<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Context" data-toc-modified-id="Context-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Context</a></span></li><li><span><a href="#Library" data-toc-modified-id="Library-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Library</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Date" data-toc-modified-id="Date-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Date</a></span></li><li><span><a href="#Address" data-toc-modified-id="Address-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Address</a></span></li></ul></li><li><span><a href="#Holidays-data" data-toc-modified-id="Holidays-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Holidays data</a></span></li><li><span><a href="#Calculating-delivery-days" data-toc-modified-id="Calculating-delivery-days-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Calculating delivery days</a></span></li><li><span><a href="#Calculating-supposed-delivery-days" data-toc-modified-id="Calculating-supposed-delivery-days-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Calculating supposed delivery days</a></span></li><li><span><a href="#Calculating-late-orders" data-toc-modified-id="Calculating-late-orders-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Calculating late orders</a></span><ul class="toc-item"><li><span><a href="#Case-1:-second-delivery-later-than-3-days" data-toc-modified-id="Case-1:-second-delivery-later-than-3-days-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Case 1: second delivery later than 3 days</a></span></li><li><span><a href="#Case-2:-first-delivery-later-than-supposed-delivery-date" data-toc-modified-id="Case-2:-first-delivery-later-than-supposed-delivery-date-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Case 2: first delivery later than supposed delivery date</a></span></li><li><span><a href="#Combining-case-1-and-case-2" data-toc-modified-id="Combining-case-1-and-case-2-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Combining case 1 and case 2</a></span></li></ul></li><li><span><a href="#Submission" data-toc-modified-id="Submission-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Submission</a></span></li></ul></div>

# Context

Logistic is one of the most important factor affecting the customers' satisfaction. During current COVID situation, we experience the impact logistic disruption has on all of us.

As part of the Shopee Code League competition, Shopee set a challenge for us to investigate their logistic provider's quality.

Shopee defined a good quality logistic provider by monitoring the days taken for a delivery. Logistic providers signed Service Level Agreements (SLA) with Shopee and orders that are fulfilled later than the stated duration in SLA is deemed late. Logistic providers will also be served penalty to ensure they uphold the quality of Shopee.

We are given one dataframe containing the delivery dates and locations, and we are tasked to investigate the late orders.

Original dataset and competition detail is available at: https://www.kaggle.com/c/logistics-shopee-code-league/overview

# Library

In [1]:
import numpy as np #numeric operations
import pandas as pd #dataframe
import matplotlib.pyplot as plt #for plotting
import datetime #for datetime operations

# Load data

In [2]:
data = pd.read_csv('datasets/delivery_orders_march.csv')

We certainly need to clean up the data before analysing it. We will convert the Unix time stamp into datetime objects for easy interpretation and extract out the cities from the addresses for us to compare the orders delivery schedules.

In [3]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   orderid              int64  
 1   pick                 int64  
 2   1st_deliver_attempt  float64
 3   2nd_deliver_attempt  float64
 4   buyeraddress         object 
 5   selleraddress        object 
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


# Preprocessing

Date

- Convert 1st_deliver_attempt to datetime
- Convert 2nd_deliver_attempt to datetime
- Convert pick to datetime
- extract the year, month, day only


Address
- Extract location from buyeraddress
- Extract location from selleraddress

## Date

1. Convert the UNIX timestamp into datetime object
2. Resample datetime by days

In [4]:
#example
datetime.datetime.fromtimestamp(1583137548)

datetime.datetime(2020, 3, 2, 16, 25, 48)

In [5]:
data['pick'] = data['pick'].apply(datetime.datetime.fromtimestamp) #convert into datetime object
data['pick'] = data['pick'].dt.to_period('D') #resample to days

In [6]:
data['1st_deliver_attempt'] = data['1st_deliver_attempt'].apply(datetime.datetime.fromtimestamp) #convert into datetime object
data['1st_deliver_attempt'] = data['1st_deliver_attempt'].dt.to_period('D') #resample to days

We fill missing values with 0, translate to datetime.datetime(1970, 1, 1, 7, 30)

In [7]:
data['2nd_deliver_attempt'] = data['2nd_deliver_attempt'].fillna(0)

In [8]:
data['2nd_deliver_attempt'] = data['2nd_deliver_attempt'].apply(datetime.datetime.fromtimestamp) #convert into datetime object
data['2nd_deliver_attempt'] = data['2nd_deliver_attempt'].dt.to_period('D') #resample to days

## Address

1. Extract city from address

In [9]:
data['buyeraddress'] = data['buyeraddress'].\
                     where(~data['buyeraddress'].str.lower().str.contains('metro manila$'),'Metro Manila').\
                     where(~data['buyeraddress'].str.lower().str.contains('luzon$'),'Luzon').\
                     where(~data['buyeraddress'].str.lower().str.contains('visayas$'),'Visayas').\
                     where(~data['buyeraddress'].str.lower().str.contains('mindanao$'),'Mindanao')

In [10]:
data['buyeraddress'].value_counts()

Metro Manila    1560698
Luzon            804888
Visayas          479595
Mindanao         331132
Name: buyeraddress, dtype: int64

In [11]:
data['selleraddress'] = data['selleraddress'].\
                     where(~data['selleraddress'].str.lower().str.contains('metro manila$'),'Metro Manila').\
                     where(~data['selleraddress'].str.lower().str.contains('luzon$'),'Luzon').\
                     where(~data['selleraddress'].str.lower().str.contains('visayas$'),'Visayas').\
                     where(~data['selleraddress'].str.lower().str.contains('mindanao$'),'Mindanao')

In [12]:
data['selleraddress'].value_counts()

Metro Manila    3176312
Luzon                 1
Name: selleraddress, dtype: int64

In [13]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,1970-01-01,Metro Manila,Metro Manila
1,2219624609,2020-03-04,2020-03-06,2020-03-10,Metro Manila,Metro Manila
2,2220979489,2020-03-04,2020-03-06,1970-01-01,Metro Manila,Metro Manila
3,2221066352,2020-03-05,2020-03-07,1970-01-01,Metro Manila,Metro Manila
4,2222478803,2020-03-04,2020-03-06,1970-01-01,Luzon,Metro Manila


# Holidays data

Shopee has their own holiday definition for this competition.\
However, it is easy to change the holiday data and include other holidays.

In [14]:
holiday_data = ['2020-03-08','2020-03-25','2020-03-30','2020-03-31']

In [15]:
holiday_data

['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']

# Calculating delivery days

We will be calculating the number of delivery days between 
1. order day to 1st delivery
2. 1st delivery to 2nd delivery date.

In [16]:
#extract out the dates and convert into numpy format for fast computation
pick_df = data['pick'].values.astype('datetime64[D]')
deliver_1st = data['1st_deliver_attempt'].values.astype('datetime64[D]')
deliver_2nd = data['2nd_deliver_attempt'].values.astype('datetime64[D]')

In [17]:
#calculate the working days between the dates
deliver_days = np.busday_count(pick_df,deliver_1st, weekmask='1111110', holidays=holiday_data)
deliver_2nd_days = np.busday_count(deliver_1st,deliver_2nd, weekmask='1111110', holidays=holiday_data)

In [18]:
#correct the missing days
deliver_2nd_days = np.where(deliver_2nd_days<0, 0, deliver_2nd_days)

In [19]:
#add into data df
data['1st_deliver_days'] = deliver_days
data['2nd_deliver_days'] = deliver_2nd_days

In [20]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,1st_deliver_days,2nd_deliver_days
0,2215676524,2020-03-02,2020-03-05,1970-01-01,Metro Manila,Metro Manila,3,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,Metro Manila,Metro Manila,2,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,Metro Manila,Metro Manila,2,0
3,2221066352,2020-03-05,2020-03-07,1970-01-01,Metro Manila,Metro Manila,2,0
4,2222478803,2020-03-04,2020-03-06,1970-01-01,Luzon,Metro Manila,2,0


# Calculating supposed delivery days

We will be calculating the delivery days according to SLA

In [21]:
#from buyer to seller
delivery_data = {'Metro Manila':{'Metro Manila':3, 'Luzon':5, 'Visayas':7, 'Mindanao':7},
                 'Luzon':       {'Metro Manila':5, 'Luzon':5, 'Visayas':7, 'Mindanao':7},
                 'Visayas':     {'Metro Manila':7, 'Luzon':7, 'Visayas':7, 'Mindanao':7},
                 'Mindanao':    {'Metro Manila':7, 'Luzon':7, 'Visayas':7, 'Mindanao':7}}

In [22]:
delivery_data

{'Metro Manila': {'Metro Manila': 3, 'Luzon': 5, 'Visayas': 7, 'Mindanao': 7},
 'Luzon': {'Metro Manila': 5, 'Luzon': 5, 'Visayas': 7, 'Mindanao': 7},
 'Visayas': {'Metro Manila': 7, 'Luzon': 7, 'Visayas': 7, 'Mindanao': 7},
 'Mindanao': {'Metro Manila': 7, 'Luzon': 7, 'Visayas': 7, 'Mindanao': 7}}

In [23]:
#convert into numpy format for speed computation
buyer_seller_add = data[['buyeraddress','selleraddress']].values
buyer_seller_add

array([['Metro Manila', 'Metro Manila'],
       ['Metro Manila', 'Metro Manila'],
       ['Metro Manila', 'Metro Manila'],
       ...,
       ['Metro Manila', 'Metro Manila'],
       ['Luzon', 'Metro Manila'],
       ['Luzon', 'Metro Manila']], dtype=object)

In [24]:
supposed_days = list(map(lambda x: delivery_data[x[0]][x[1]], buyer_seller_add))
supposed_days[:5]

[3, 3, 3, 3, 5]

In [25]:
#add into data df
data['supposed_days'] = supposed_days

In [26]:
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,1st_deliver_days,2nd_deliver_days,supposed_days
0,2215676524,2020-03-02,2020-03-05,1970-01-01,Metro Manila,Metro Manila,3,0,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,Metro Manila,Metro Manila,2,3,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,Metro Manila,Metro Manila,2,0,3
3,2221066352,2020-03-05,2020-03-07,1970-01-01,Metro Manila,Metro Manila,2,0,3
4,2222478803,2020-03-04,2020-03-06,1970-01-01,Luzon,Metro Manila,2,0,5


# Calculating late orders

We will be considering the orders that are late based on two criteria:
1. The second delivery is later than 3 days
2. The first delivery is later than the days agreed in SLA

## Case 1: second delivery later than 3 days

Second delivery means the delivery done after a failed first delivery. In this competition Shopee assumes all goods will be delivered by second delivery.\
Shopee require all second delivery to be within 3 days.

In [27]:
data['2nd_deliver_late'] = np.where(data['2nd_deliver_days']>3, True, False)

In [28]:
data['2nd_deliver_late'].value_counts()

False    2830684
True      345629
Name: 2nd_deliver_late, dtype: int64

## Case 2: first delivery later than supposed delivery date

Orders that are delivered later than Shopee's SLA with logistic providers are considered late orders.

In [29]:
data['1st_deliver_late'] = np.where(data['1st_deliver_days']>data['supposed_days'], True, False)

In [30]:
data['1st_deliver_late'].value_counts()

False    2734919
True      441394
Name: 1st_deliver_late, dtype: int64

## Combining case 1 and case 2

Since the definition of late orders is either the order is delivered later than 3 days for second delivery AND order is delivered later than the agreed days in SLA, we will be combining the two cases.

In [31]:
data['is_late'] = np.where(data['1st_deliver_late'] | data['2nd_deliver_late'], 1, 0)

In [32]:
data['is_late'].value_counts()

0    2413891
1     762422
Name: is_late, dtype: int64

In [33]:
data[data['is_late'] == 1].head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,1st_deliver_days,2nd_deliver_days,supposed_days,2nd_deliver_late,1st_deliver_late,is_late
7,2224695304,2020-03-02,2020-03-10,1970-01-01,Metro Manila,Metro Manila,7,0,3,False,True,1
19,2227790841,2020-03-02,2020-03-10,2020-03-18,Mindanao,Metro Manila,7,7,7,True,False,1
21,2227816770,2020-03-02,2020-03-06,2020-03-07,Metro Manila,Metro Manila,4,1,3,False,True,1
22,2227817800,2020-03-02,2020-03-11,1970-01-01,Visayas,Metro Manila,8,0,7,False,True,1
24,2227831996,2020-03-02,2020-03-06,2020-03-10,Metro Manila,Metro Manila,4,3,3,False,True,1


# Submission

In [34]:
submission = data[['orderid','is_late']]

In [35]:
submission.info()
submission.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   orderid  int64
 1   is_late  int64
dtypes: int64(2)
memory usage: 48.5 MB


,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0


In [36]:
submission.to_csv('submission3.csv',index=False)